# 요약

- 개요
    - 주식의 가격에는 다양한 요소들이 영향을 미치지만 그 중 뉴스에 민감하다고 판단하여 뉴스 기사의 제목을 분석하여 긍정/부정 평가를 한다.
1. 데이터 수집
    - 최근 1일, 정확도 순서로 검색어 입력 시 그에 대한 뉴스 제목 정보를 크롤링한다.
2. 전처리
    - konlpy의 Hannanum으로 제목에 대해서 형태소 분석을 하고 tokenize를 진행한다.
3. 모델링
    - pre-train 된 한국식 bert 모델인 koBart 모델을 사용하여 sentiment(감성) 점수를 부여한다.
    - 제목을 수치화한 'sentiment' 값을 얻어내고 0.5 기준으로 이상이면 긍정, 미만이면 부정으로 'label' 이라고 labeling을 수행
4. 결과
    - 두 가지 방식으로 긍/부정을 결론 짓는다.
        - 모든 뉴스의 감성 점수에 대한 평균을 기준으로 0.5 미만이면 부정/ 0.5 이상이면 긍정
        - 각각의 뉴스에 대해 미리 긍/부정을 나눈 'label'의 개수에 대해 긍정 뉴스가 많으면 긍정, 부정 뉴스가 많으면 부정으로 결론짓는다.

# 네이버 뉴스 크롤링
### 1. 데이터 수집
- 네이버 검색창에 키워드에 대한 뉴스를 크롤링함
- 정확도순으로 최근 1일의 뉴스 데이터를 100개 이하로 수집
- requests 라이브러리로 url에 접근하고 BeautifulSoup으로 뉴스 기사 본문을 파싱하지만 기사 제목 수준으로만 감성 분성을 진행함

In [1]:
# 크롤링시 필요한 라이브러리 불러오기
from bs4 import BeautifulSoup
import requests
import re
import datetime
from tqdm import tqdm
import pandas as pd

# 페이지 입력 (1 페이지당 기사 10개 이하)
def makePgNum(num):
    if num == 1:
        return num
    elif num == 0:
        return num + 1
    else:
        return num + 9 * (num - 1)


# search : 검색어, pd=4 : 최근 1일, start_page : 몇 페이지
def makeUrl(search, start_pg, end_pg):
    if start_pg == end_pg:
        start_page = makePgNum(start_pg)
        # 정확도순(디폴트)으로 1일간의 뉴스(pd=4) 
        url = "https://search.naver.com/search.naver?where=news&sm=tab_pge&query=" + search +"&start=" + str(
            start_page)
        
        return url
    else:
        # url 부분에서 정확도순서로 1일 데이터를 분류 가능
        urls = []
        for i in range(start_pg, end_pg + 1):
            page = makePgNum(i)
            
            url = "https://search.naver.com/search.naver?where=news&sm=tab_pge&query=" + search +"&pd=4"+"&start=" + str(page)
            #url = "https://search.naver.com/search.naver?where=news&query=%EC%B9%B4%EC%B9%B4%EC%98%A4&sm=tab_opt&sort=0&photo=0&field=0&pd=3&ds=2021.09.10&de=2021.09.15&docid=&related=0&mynews=0&office_type=0&office_section_code=0&news_office_checked=&nso=so%3Ar%2Cp%3Afrom20210910to20210915&is_sug_officeid=0"+"&start=" + str(page)
            urls.append(url)
        return urls

    # html에서 원하는 속성 추출하는 함수 만들기 (기사, 추출하려는 속성값)

# 기사 내용 크롤링 함수
def news_attrs_crawler(articles, attrs):
    attrs_content = []
    for i in articles:
        attrs_content.append(i.attrs[attrs])
    return attrs_content


# ConnectionError방지
headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) Chrome/98.0.4758.102"}

# html생성해서 기사크롤링하는 함수 만들기(url): 링크를 반환
def articles_crawler(url):
    # html 불러오기
    original_html = requests.get(i, headers=headers)
    html = BeautifulSoup(original_html.text, "html.parser")

    url_naver = html.select(
        "div.group_news > ul.list_news > li div.news_area > div.news_info > div.info_group > a.info")
    url = news_attrs_crawler(url_naver, 'href')
    return url



#### 입력부

In [2]:
#####뉴스크롤링 시작#####

# 검색어 입력
search = input("검색 키워드 입력 : ")
# 검색 시작할 페이지 입력
page = 1
# 검색 종료할 페이지 입력
page2 = 10

searches = ['삼성전자','LG에너지솔루션','SK하이닉스','삼성바이오로직스','POSCO홀딩스',"LG화학","삼성SDI","현대차","NAVER"]

# naver url 생성
url = makeUrl(search, page, page2)

# 뉴스 크롤러 실행
news_titles = []
news_url = []
news_contents = []
news_dates = []

for i in url:
    url = articles_crawler(url)
    news_url.append(url)


# 제목, 링크, 내용 1차원 리스트로 꺼내는 함수 생성
def makeList(newlist, content):
    for i in content:
        for j in i:
            newlist.append(j)
    return newlist


# 제목, 링크, 내용 담을 리스트 생성
news_url_1 = []

# 1차원 리스트로 만들기(내용 제외)
makeList(news_url_1, news_url)

# NAVER 뉴스만 남기기
final_urls = []
for i in tqdm(range(len(news_url_1))):
    if "news.naver.com" in news_url_1[i]:
        final_urls.append(news_url_1[i])
    else:
        pass

# 뉴스 내용 크롤링
for i in tqdm(final_urls):
    # 각 기사 html get하기
    news = requests.get(i, headers=headers)
    news_html = BeautifulSoup(news.text, "html.parser")

    # 뉴스 제목 가져오기
    title = news_html.select_one("#ct > div.media_end_head.go_trans > div.media_end_head_title > h2")
    if title == None:
        title = news_html.select_one("#content > div.end_ct > div > h2")
# ------------------------------------------------------------------------------------------------------------------
    # 뉴스 본문 가져오기 (일단 구현은 해놓음 but 일단 기사 제목 수준에서 진행)
    content = news_html.select("div#dic_area")
    if content == []:
        content = news_html.select("#articeBody")
    content = ''.join(str(content))

    # html태그제거 및 텍스트 다듬기
    pattern1 = '<[^>]*>'
    title = re.sub(pattern=pattern1, repl='', string=str(title))
    content = re.sub(pattern=pattern1, repl='', string=content)
    pattern2 = """[\n\n\n\n\n// flash 오류를 우회하기 위한 함수 추가\nfunction _flash_removeCallback() {}"""
    content = content.replace(pattern2, '')
    
    news_titles.append(title)
    news_contents.append(content)

    try:
        html_date = news_html.select_one(
            "div#ct> div.media_end_head.go_trans > div.media_end_head_info.nv_notrans > div.media_end_head_info_datestamp > div > span")
        news_date = html_date.attrs['data-date-time']
    except AttributeError:
        news_date = news_html.select_one("#content > div.end_ct > div > div.article_info > span > em")
        news_date = re.sub(pattern=pattern1, repl='', string=str(news_date))
        news_date = news_date[0:10]
    # 날짜 가져오기
    news_dates.append(news_date)

print("\n[뉴스 제목]")
print(news_titles)
print("\n[뉴스 링크]")
print(final_urls)
print("\n[뉴스 내용]")
print(news_contents)
print("\n[뉴스 날짜]")
print(news_dates)

print('news_title: ', len(news_titles))
print('news_url: ', len(final_urls))
print('news_contents: ', len(news_contents))
print('news_dates: ', len(news_dates))

검색 키워드 입력 : 삼성전자


100%|██████████████████████████████████████████████████████████████████████████████████| 61/61 [00:08<00:00,  6.83it/s]


[뉴스 제목]
["[단독]경계현 등 삼성전자 경영진, '반도체 특성화대' 서울대 총출동", '삼성전자서비스, 수해 복구 특별점검 서비스 실시', '에코프로 대신 삼성전자 사들인 개미…증권가 “9만5000원 간다”', '삼성전자, 갤S23 시리즈 대상 One UI 6 베타 시작', '이재용 복권 1年…삼성전자 주가도 10% 넘게 올랐네 [투자360]', '안방부터 지킨다? 삼성전자가 한국서 폴더블폰 가장 싸게 파는 까닭은', '공정위, ‘삼성전자에 갑질’ 美 브로드컴 제재 9월 결정', '침수 세탁기 점검하는 삼성전자서비스', '[데이터로 보는 증시]호텔신라·삼성전자, 기관·외국인 주간 코스피 순매수 1위(8월 7일~11일)', '尹, 오늘 광복절 특사 확정…최지성·장충기 제외, 김태우 포함', "큰손 '연기금' 삼전·포스코 팔고 담은 종목은", "삼성, 中 스마트폰시장 부활 날개 '폈다'", '尹 광복절특사..재계·김태우 사면', "HBM3가 쏘아올린 차세대 메모리 혈투..삼성·SK '2파전' 압축", '아이폰15 성능 얼마나 뛸까…벤치마크 유출 점수 보니', '이재용, 글로벌 광폭 행보 1년…삼성의 새 먹거리 탐색', '낸드플래시, 앞이 캄캄하다[ICT]', "동부건설, '용인 센트레빌 그리니에' 이달 중 분양", "에코프로 쏠림 진정… 개미 '삼전·포스코·2차전지' 분산투자", '오늘의집, 연중 최대 세일 \'오시즌위크\'…"89%까지 할인"', "정부 육성 의지에…반도체과 합격선 의대 '턱밑'", '中, 90형대 초대형 TV도 반값 파상공세', '삼성부터 한화까지...대기업 매료시킨 200兆 로봇시장', "[칩톡]'개봉박두' 美 반도체 보조금 69兆 잡아라", '[단독]尹, 초기부터 \'김태우 사면\' 의지…"권력형 비리 공익신고자"', '증시 변동성 확대 진원지 이차전지주 쏠림현상 완화', '“한국기업 오면 인센티브”…인구 대국이 러브콜 한 회사는 어디', '“남자와 여자의 눈 다르다” 밝혀낸 이창림 교수, 젊은 과학자상 탔다', '올 기업실적 상저하

In [9]:
news_df = pd.DataFrame({'date': news_dates, 'title': news_titles, 'content' : news_contents})
news_df

,date,title,content
0,2023-08-14 05:31:05,"[단독]경계현 등 삼성전자 경영진, '반도체 특성화대' 서울대 총출동",[]
1,2023-08-14 11:16:36,"삼성전자서비스, 수해 복구 특별점검 서비스 실시",[]
2,2023-08-14 06:02:01,에코프로 대신 삼성전자 사들인 개미…증권가 “9만5000원 간다”,[]
3,2023-08-13 17:56:53,"삼성전자, 갤S23 시리즈 대상 One UI 6 베타 시작",[]
4,2023-08-14 08:04:01,이재용 복권 1年…삼성전자 주가도 10% 넘게 올랐네 [투자360],[]
...,...,...,...
56,2023-08-14 08:48:16,"한미글로벌, 상반기 매출 2천56억원 역대 최대…영업익 137억원",[]
57,2023-08-14 11:08:03,“K반도체에 유럽 공장 필요해질 것” [美반도체법 시행 1년],[]
58,2023-08-14 03:05:30,더 작고 더 빠르게… 차량용 반도체 ‘초미세공정’ 경쟁,[]
59,None,None,[]


### 2. 데이터 전처리 및 모델 적용
- 제목을 단어 단위로 토큰화를 수행함.
    - konlpy의 Hannanum을 선택함.
- 토큰화된 제목을 koBart 모델을 사용하여 감성 수치를 추출하여 'sentiment'로 저장한다.
- sentiment 값을 토대로 0.5 이상이면 1, 0.5 미만이면 0으로 하여 'label'로 저장한다.

In [4]:
from konlpy.tag import Hannanum  # Hannanum 형태소 분석기 불러오기
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification

df = news_df

# KoELECTRA 모델 로드
model_name = "hyunwoongko/kobart"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# 형태소 분석 함수
hannanum = Hannanum()  # Hannanum 형태소 분석기 객체 생성
def tokenize(text):
    return hannanum.morphs(text)

# title 열에 대해 형태소 분석 적용

df['title'] = df['title'].astype(str).apply(tokenize)
# for i in range(df.shape[0]):
#     df.iloc[i,1] = df.iloc[i,1].apply(tokenize)


# 감성 분석을 위한 전처리 함수
def preprocess(text):
    inputs = tokenizer(text, padding=True, truncation=True, return_tensors="pt")
    inputs.to(device)
    return inputs

# 예측 함수
def predict(inputs):
    outputs = model(**inputs)
    logits = outputs.logits
    probs = logits.softmax(dim=-1)
    return probs[0].detach().cpu().numpy()

# title을 예측해서 수치화시켜 sentiment으로 저장. 즉, title를 수치화 시킨 것이 sentiment
df['sentiment'] = df['title'].apply(lambda x: predict(preprocess(' '.join(x))))

# 0.5 기준으로 하면 부정확하긴 함... 정확도를 높이려면 이 부분 건들면 좋을 듯 또는 중립을 포함시키는 것도 해봐야 할 듯
def convert_sentiment(probs):
    if probs[0] < 0.5:
        return 0
    elif probs[0]>= 0.5:
        return 1
#     else:
#         return '중립'

# train할 label은 제목을 읽고 내가 직접 라벨링
# test할 label은 0.5를 기준으로 sentiment가 0.5보다 크면 1, 작으면 0으로 기준 세움
df['label'] = df['sentiment'].apply(convert_sentiment)
df['sentiment'] = df['sentiment'].apply(lambda x: x[0]).tolist()
df=round(df,2)

You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels wil be overwritten to 2.
Some weights of BartForSequenceClassification were not initialized from the model checkpoint at hyunwoongko/kobart and are newly initialized: ['classification_head.dense.weight', 'classification_head.dense.bias', 'classification_head.out_proj.weight', 'classification_head.out_proj.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


In [5]:
df

,date,title,content,sentiment,label
0,2023-08-14 05:31:05,"[[, 단독, ], 경계현, 등, 삼성전자, 경영진, ,, ', 반도체, 특성화대,...",[],0.64,1
1,2023-08-14 11:16:36,"[삼성전자서비스, ,, 수하, 어, 복구, 특별점검, 서비스, 실시]",[],0.48,0
2,2023-08-14 06:02:01,"[에코프, 로, 대, 이, 시ㄴ, 삼성전자, 사, 아, 들이, ㄴ, 개미…증권, 가...",[],0.49,0
3,2023-08-13 17:56:53,"[삼성전자, ,, 갤S23, 시리즈, 대상, One, UI, 6, 베, 어, 타, ...",[],0.55,1
4,2023-08-14 08:04:01,"[이재용, 복권, 1年…삼성전자, 주가, 도, 10, %, 넘, 게, 오르, 아네,...",[],0.51,1
...,...,...,...,...,...
56,2023-08-14 08:48:16,"[한미글로벌,, 상반기, 매출, 2천56억원, 역대, 최대…영업익, 137억원]",[],0.60,1
57,2023-08-14 11:08:03,"[“K반도체, 에, 유럽, 공장, 필요, 하, 어, 지, ㄹ, 것”, [, 美반도체...",[],0.65,1
58,2023-08-14 03:05:30,"[더, 작, 고, 더, 빠르게…, 차량용, 반도체, ‘초미세공정’, 경쟁]",[],0.69,1
59,None,[None],[],0.58,1


### 3. 결과
#### 1) 평균으로 분석
- 뉴스 기사의 감성 점수의 평균이 0.5 이상이면 다음 날의 주가의 상승을 예측
- 반대의 경우 하락으로 예측함

In [6]:
print(f"모든 뉴스에 대한 평균 감성 점수 값: {df['sentiment'].mean()}")
if df['sentiment'].mean() >= 0.5:
    print("주가의 상승 예측")
else:
    print("주가의 하락 예측")

모든 뉴스에 대한 평균 감성 점수 값: 0.539672131147541
주가의 상승 예측


#### 2) 개수로 분석
- label의 값이 1인 경우의 뉴스가 더 많은 경우에 다음날의 주가의 상승을 예측
- 반대의 경우 하락을 예측함.

In [7]:
sentiment = 0
print(f"긍정 뉴스의 개수: {df[df['label'] == 1].label.count()}")
print(f"부정 뉴스의 개수: {df[df['label'] == 0].label.count()}")

if df[df['label'] == 1].label.count() > df[df['label'] == 0].label.count():
    sentiment = 1
    print("주가 상승으로 예측")
else:
    sentiment = 0
    print("주가 하락으로 예측")



긍정 뉴스의 개수: 38
부정 뉴스의 개수: 23
주가 상승으로 예측


In [23]:
from collections import Counter

sentimented_title = []
print(len(df['title']))
for i in range(len(df['title'])):
    for j in range(len(df['title'][i])):
        if len(df['title'][i][j]) != 1: 
            sentimented_title.append(df['title'][i][j])

counter = Counter(sentimented_title)
counter

61


['단독',
 '경계현',
 '삼성전자',
 '경영진',
 '반도체',
 '특성화대',
 '서울대',
 '총출동',
 '삼성전자서비스',
 '수하',
 '복구',
 '특별점검',
 '서비스',
 '실시',
 '에코프',
 '시ㄴ',
 '삼성전자',
 '들이',
 '개미…증권',
 '“9만5000원',
 '간다”',
 '삼성전자',
 '갤S23',
 '시리즈',
 '대상',
 'One',
 'UI',
 '시작',
 '이재용',
 '복권',
 '1年…삼성전자',
 '주가',
 '10',
 '오르',
 '아네',
 '[투자360',
 '안방',
 '부터',
 '지키',
 'ㄴ다',
 '삼성전자',
 '한국서',
 '폴더블폰',
 '가장',
 '까닭',
 '공정위',
 '‘삼성전자',
 '갑질’',
 '브로드컴',
 '제재',
 '9월',
 '결정',
 '침수',
 '세탁기',
 '점검',
 '삼성전자서비스',
 '데이터',
 '증시]호텔신라·삼성전자,',
 '기관·외국',
 '주간',
 '코스피',
 '순매수',
 '8월',
 '7일',
 '11일',
 '오늘',
 '광복절',
 '특사',
 '확정…최지성·장충기',
 '제외',
 '김태우',
 '포함',
 '큰손',
 '연기금',
 '삼전·포스코',
 '종목',
 '삼성',
 '스마트폰시장',
 '부활',
 '날개',
 '었다',
 '광복절특사',
 '..',
 '재계·김태우',
 '사면',
 'HBM3',
 '쏘아올린',
 '차세대',
 '메모리',
 '혈투',
 '..',
 '삼성·SK',
 '2파',
 '압축',
 '아이폰15',
 '성능',
 '얼마나',
 '뛸까…벤치마크',
 '유출',
 '점수',
 '이재',
 '글로벌',
 '광폭',
 '행보',
 '1년…삼성',
 '먹거리',
 '탐색',
 '낸드플래시,',
 '캄캄하다',
 'ICT',
 '동부건설',
 '센트레빌',
 "그리니에'",
 '이달',
 '분양',
 '에코프',
 '쏠리',
 '진정…',
 '개미',
 "'삼전·포스코·2차전지'",
 '

In [33]:
counter.most_common(10)

[('삼성', 5),
 ('삼성전자', 4),
 ('반도체', 4),
 ('복권', 4),
 ('...', 4),
 ('단독', 3),
 ('부터', 3),
 ('광복절', 3),
 ('었다', 3),
 ('삼성전자서비스', 2)]

### csv로 만들면서 마무리

In [8]:
###데이터 프레임으로 만들기###

# 데이터 프레임 만들기
# news_df = pd.DataFrame({'date': news_dates, 'title': news_titles, 'link': final_urls, 'content': news_contents})

# 데이터 프레임 저장
now = datetime.datetime.now()
date = str(now.year%100)+format(now.month,'02')+format(now.day,'02')
df.to_csv(f'csv/{date}_{search}_감성분석.csv', encoding='utf-8-sig', index=False)

단어 개수 세기